In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 11.5 MB/s 
     |████████████████████████████████| 7.6 MB 13.7 MB/s 
     |████████████████████████████████| 182 kB 72.1 MB/s 


#Importing libraries

In [4]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report,accuracy_score

import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

#Reading dataset

In [13]:
df=pd.read_csv('/content/drive/MyDrive/NLP /Project/H2_Hate_Offensive_Language_Identification_train.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7593 entries, 0 to 7592
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7593 non-null   object
 1   label   7593 non-null   object
 2   id      7593 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 178.1+ KB


In [15]:
df['label'].unique()

array(['NOT', 'HOF'], dtype=object)

In [16]:
df

,text,label,id
0,"ऑटोरिक्शा चालक जावेद खान कहते हैं, ""कोविड-19 क...",NOT,0
1,@Raghuveerdutt62 @Arunesh24797107 रघुवीर जी यह...,NOT,1
2,@Drmahera552 @common000786 मोदी तुम इस्तीफा दो...,NOT,2
3,@sab_hateDMS Does she wear mesh hooker hose an...,HOF,3
4,What a bunch of absolute fucking idiots in #in...,HOF,4
...,...,...,...
7588,@NewsNationTV दीपक जी! आज के विपक्षी दल जनता क...,HOF,7588
7589,@pranaypiyush @ndtv That not gonna help india ...,NOT,7589
7590,@ABPNews इसमें विवादित क्या है ? ताड़का बेहतर ...,HOF,7590
7591,मक़तल में आ गया हूं तेरा साथ चाहता हूं तदफी़न ...,NOT,7591


In [17]:
df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes
df['label'] = df['label'].astype(np.int64)

In [33]:
df

,text,label,id
0,"ऑटोरिक्शा चालक जावेद खान कहते हैं, ""कोविड-19 क...",1,0
1,@Raghuveerdutt62 @Arunesh24797107 रघुवीर जी यह...,1,1
2,@Drmahera552 @common000786 मोदी तुम इस्तीफा दो...,1,2
3,@sab_hateDMS Does she wear mesh hooker hose an...,0,3
4,What a bunch of absolute fucking idiots in #in...,0,4
...,...,...,...
7588,@NewsNationTV दीपक जी! आज के विपक्षी दल जनता क...,0,7588
7589,@pranaypiyush @ndtv That not gonna help india ...,1,7589
7590,@ABPNews इसमें विवादित क्या है ? ताड़का बेहतर ...,0,7590
7591,मक़तल में आ गया हूं तेरा साथ चाहता हूं तदफी़न ...,1,7591


In [18]:
X = list(df['text'].values)
y = list(df['label'].values)

In [19]:
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size = 0.10, random_state=42,stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_tmp, y_tmp, test_size = 0.111, random_state=42,stratify=y_tmp)

print(len(X_train))
print(len(X_valid))
print(len(X_test))

6074
759
760


In [11]:
print(len(X))

7593


##Pre-processing the whole dataset

In [3]:
df['text'] = df.text.astype(str)

In [4]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 4.9 MB/s 
     |████████████████████████████████| 106 kB 52.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import contractions

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import spacy
nlp = spacy.load("en_core_web_sm")

from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer,TreebankWordTokenizer
from nltk.tokenize import wordpunct_tokenize,TweetTokenizer



stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')
def autospell(text):
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 5.4 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=b5e891a6e6282d19e8cb13763e5b1fac9cec46d6355e786237a50d575f726cd3
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [ ]:
def pos_tagger(tag):
    nltk_tag=tag[1]
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV   
    return 'n'

In [ ]:
stop_list=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves',\
     'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', \
     'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',\
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'or', 'because', 'as', 'until','of', 'at', 'by', 'for', 'with', 'about', 'between',\
    'into', 'through', 'during', 'to', 'from','in', 'out','on','again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 
    'just','should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y','u','im','go','will','come','whats','twitter','tweet',\
    'know','x','yeah','year','yet','youre','would','do','can','nan','see','look','one','could','तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']

stop_reg = r'\b(?:'+'|'.join(stop_list)+r')\b'
stop_reg
stopword_finder= re.compile(stop_reg)

In [ ]:
def processing(sentence, verbose=False):
  og_sentence = sentence
  sentence=sentence.lower()#CASE FOLDING
  html_finder = re.compile(r'<.*?>|&\w+;')
  rmhtml_Text=html_finder.sub('',sentence)  #remove HTML Tags
  # rmurl_Text = re.sub(r'http\S+|https:\S+|www\S+|.com\S+', '', rmhtml_Text) #remove URL tags
  pre_url = r'(?:https?://)|(?:www\.)'
  url_set = r'[\w+=?/._\-:;&*^$#@~`!|]'
  rmurl_Text=re.sub(f'(?:{pre_url}){url_set}+|{url_set}+\.com{url_set}*', '',rmhtml_Text)
  username_finder = re.compile('@\w+')
  rmurl_Text=username_finder.sub('',rmurl_Text) #remove usernames
  rmwhite_Text=re.sub(' +', ' ',rmurl_Text) #remove extra whitespace
  rmwhite_Text = contractions.fix(rmwhite_Text) #Remove contractions
  rmpunt_text=re.sub('[^\w\s]','',rmwhite_Text) #punctuation and username removal
  
  token_text=[token for token in TweetTokenizer().tokenize(rmpunt_text)]#tokenize text 
  
  # stem_text= [stemmer.stem(word) for word in rmpunt_text]#stemming
  pos_tags=[pos_tagger(tag) for tag in nltk.pos_tag(token_text)]
  lem_text= [lemmatizer.lemmatize(word,pos=pos) for word,pos in zip(token_text,pos_tags)] #lemmatize word
  # print(lem_text)
  rmstop_text = [tok for tok in lem_text if not stopword_finder.search(tok)]
  # print(rmstop_text)
  sentence=TreebankWordDetokenizer().detokenize(rmstop_text) #convert to sentence 
  spell_sentence = autospell(sentence)
  if verbose:
    print(f'Original sentence :\n{og_sentence}\n')
    print(f'Removing HTML and (Lowercasing):\n{rmhtml_Text}\n')
    print(f'Removing URL tags (and usernames):\n{rmurl_Text}\n')
    print(f'Removing whitespace(and expand contractions):\n{rmwhite_Text}\n')
    print(f'Remove punctuations:\n{rmpunt_text}\n')   
    print(f'Tokenize Text:\n{token_text}\n')
    print(f'Lemmatize Text:\n{lem_text}\n')
    print(f'Removing stop words Text:\n{rmstop_text}\n')
    print(f'Spelling Correction:\n{spell_sentence}\n')
  
  return spell_sentence

In [ ]:
text = "Stephanie Pratt Tells MTV News 'The Hills' Did Not Make Me Bulimic' \
    http://bit.ly/n4wL4 BETTER but still > half the story is missing" 
text=df['text'][0]

processing(text,verbose=True)

Original sentence :
ऑटोरिक्शा चालक जावेद खान कहते हैं, "कोविड-19 का गंभीर मरीज बिना ऑक्सीजन सपोर्ट के अस्पताल नहीं लाया जा सकता है. इसलिए मैंने सोचा ऑटोरिक्शा को एंबुलेंस में क्यों ना बदल डालूं." #IndiaCovidCrisis #OxygenCrisis   https://t.co/F1snlleyMP

Removing HTML and (Lowercasing):
ऑटोरिक्शा चालक जावेद खान कहते हैं, "कोविड-19 का गंभीर मरीज बिना ऑक्सीजन सपोर्ट के अस्पताल नहीं लाया जा सकता है. इसलिए मैंने सोचा ऑटोरिक्शा को एंबुलेंस में क्यों ना बदल डालूं." #indiacovidcrisis #oxygencrisis   https://t.co/f1snlleymp

Removing URL tags (and usernames):
ऑटोरिक्शा चालक जावेद खान कहते हैं, "कोविड-19 का गंभीर मरीज बिना ऑक्सीजन सपोर्ट के अस्पताल नहीं लाया जा सकता है. इसलिए मैंने सोचा ऑटोरिक्शा को एंबुलेंस में क्यों ना बदल डालूं." #indiacovidcrisis #oxygencrisis   

Removing whitespace(and expand contractions):
ऑटोरिक्शा चालक जावेद खान कहते हैं, "कोविड-19 का गंभीर मरीज बिना ऑक्सीजन सपोर्ट के अस्पताल नहीं लाया जा सकता है. इसलिए मैंने सोचा ऑटोरिक्शा को एंबुलेंस में क्यों ना बदल डालूं." #indiaco

'ऑटरकश चलक जवद खन कहत ह कवड 19 क गभर मरज बन ऑकसजन सपरट क असपतल नह लय ज सकत ह इसलए मन सच ऑटरकश क एबलस म कय बदल डल indiacovidcrisis oxygencrisis'

In [ ]:
data=df.apply(lambda row: processing(row.text,verbose=False),axis=1)

In [ ]:
df['new_text']=data

In [ ]:
df.to_csv('/content/drive/MyDrive/NLP /Project/H2_Hate_Offensive_Language_Identification_train_processed.csv')

In [ ]:
df['new_text']

0       ऑटरकश चलक जवद खन कहत ह कवड 19 क गभर मरज बन ऑकस...
1       रघवर ज यह सतय ह क वनर कई बडबड बल वल पछ वल जनवर...
2       मद तम इसतफ द हम तमहर सथ ह resignmodi resign_pm...
3             wear mesh hooker hose phd community college
4       bunch absolute fuck idiot india indiacovidcris...
                              ...                        
7588    दपक ज आज क वपकष दल जनत क परशन करक सतत म आन चहत...
7589    not help india anyway but mean image chinesevirus
7590    इसम ववदत कय ह तडक बहतर थ जहद नह कय थ उसन उस और...
7591    मकतल म आ गय ह तर सथ चहत ह तदफन ह रह ह तर सथ चह...
7592    enough enough thanks cache din resignpmmodi no...
Name: new_text, Length: 7593, dtype: object

In [ ]:
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']

##Repo

In [11]:
model_name = 'roberta-base'
def generate_dataset(X,y, tokenizer):

  #Generate encodings
  encodings = tokenizer.batch_encode_plus(X,        
                                          truncation = True,
                                          padding=True,
                                          max_length=128,
                                          return_tensors='pt',
                                          add_special_tokens = True)

  #convert to tensor dataset
  input_ids = encodings['input_ids']
  attention_mask = encodings['attention_mask']

  input_ids = torch.tensor(input_ids)
  attention_mask = torch.tensor(attention_mask)
  y = torch.tensor(y)

  dataset = TensorDataset(input_ids,attention_mask,y)

  return dataset

In [20]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)
train_dataset = generate_dataset(X_train,y_train,tokenizer)
valid_dataset = generate_dataset(X_valid,y_valid,tokenizer)

In [14]:
def get_data_loader(dataset,sampler):
  return DataLoader(dataset=dataset,sampler=sampler(dataset),batch_size=32)

In [15]:
train_loader = get_data_loader(train_dataset,RandomSampler)
validation_loader = get_data_loader(valid_dataset,SequentialSampler)

In [16]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.cuda()
print("Imported model!")

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Imported model!


In [17]:
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-8)

num_epochs = 8
total_steps = len(train_loader) * num_epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
def compute_accuracy(preds,labels):
  preds = preds.detach().cpu().numpy()
  labels = labels.detach().cpu().numpy()
  preds = np.argmax(preds,axis=1).flatten()
  labels = labels.flatten()
  return np.sum(preds == labels)/len(labels)
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [18]:
def train():
  total_loss = 0.0
  total_acc = 0.0

  model.train()
  
  num_steps = 0

  for step,batch in enumerate(train_loader):
    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_loader)))

    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    model.zero_grad()

    outputs = model(input_ids,attention_mask = attention_mask,labels = labels.long())
    
    loss = outputs.loss
    logits = outputs.logits

    total_loss = total_loss + loss.item()
    total_acc = total_acc + compute_accuracy(logits,labels)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()

  avg_loss = total_loss/len(train_loader)
  avg_acc = total_acc/len(train_loader)

  return avg_loss,avg_acc


In [19]:
def evaluate():
  total_loss = 0.0
  total_acc = 0.0

  model.train()
  
  true_labels = []
  predictions = []

  for step,batch in enumerate(validation_loader):
    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(validation_loader)))

    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
      outputs = model(input_ids,attention_mask = attention_mask,labels = labels.long())
    
    loss = outputs.loss
    logits = outputs.logits

    total_loss = total_loss + loss.item()
    total_acc = total_acc + compute_accuracy(logits,labels)

    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits,axis=1).flatten()
    labels = labels.detach().cpu().numpy().flatten()

    true_labels.extend(labels)
    predictions.extend(logits)

  avg_loss = total_loss/len(validation_loader)
  avg_acc = total_acc/len(validation_loader)
  macro_f1_score =  classification_report(true_labels, predictions,output_dict=True)['macro avg']['f1-score']

  return avg_loss,avg_acc,macro_f1_score

In [20]:
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

best_f1 = 0.0

for epoch in range(num_epochs):
  print('\n Epoch {:} / {:}'.format(epoch + 1, num_epochs))
  
  train_loss, train_acc = train()
  valid_loss, valid_acc,f1_score = evaluate()

  train_losses.append(train_loss)
  train_accs.append(train_acc)
  valid_losses.append(valid_loss)
  valid_accs.append(valid_acc)

  if f1_score > best_f1:
      best_f1 = f1_score
      torch.save(model,'/content/drive/MyDrive/NLP /Project/final_english_model_robert_8Sept.pt')


  print(f'\nTraining Accuracy: {train_acc:.3f} | Training Loss: {train_loss:.3f} | Validation Accuracy: {valid_acc:.3f} | Validation Loss: {valid_loss:.3f} | F1 Score: {f1_score:3f}')


 Epoch 1 / 8
  Batch    50  of    190.
  Batch   100  of    190.
  Batch   150  of    190.

Training Accuracy: 0.699 | Training Loss: 0.583 | Validation Accuracy: 0.740 | Validation Loss: 0.525 | F1 Score: 0.728081

 Epoch 2 / 8
  Batch    50  of    190.
  Batch   100  of    190.
  Batch   150  of    190.

Training Accuracy: 0.779 | Training Loss: 0.473 | Validation Accuracy: 0.801 | Validation Loss: 0.472 | F1 Score: 0.797429

 Epoch 3 / 8
  Batch    50  of    190.
  Batch   100  of    190.
  Batch   150  of    190.

Training Accuracy: 0.817 | Training Loss: 0.410 | Validation Accuracy: 0.791 | Validation Loss: 0.496 | F1 Score: 0.789014

 Epoch 4 / 8
  Batch    50  of    190.
  Batch   100  of    190.
  Batch   150  of    190.

Training Accuracy: 0.845 | Training Loss: 0.358 | Validation Accuracy: 0.780 | Validation Loss: 0.542 | F1 Score: 0.777572

 Epoch 5 / 8
  Batch    50  of    190.
  Batch   100  of    190.
  Batch   150  of    190.

Training Accuracy: 0.871 | Training Loss: 0

#Eval on Test

In [21]:
test_dataset = generate_dataset(X_test,y_test,tokenizer)
test_loader = get_data_loader(test_dataset,SequentialSampler)
model = torch.load('/content/drive/MyDrive/NLP /Project/final_english_model_robert_8Sept.pt')
model = model.to(device)

In [22]:
def test_evaluate(model):
  total_loss = 0.0
  total_acc = 0.0

  model.train()
  
  true_labels = []
  predictions = []

  for step,batch in enumerate(test_loader):
    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_loader)))

    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
      outputs = model(input_ids,attention_mask = attention_mask,labels = labels.long())
    
    loss = outputs.loss
    logits = outputs.logits

    total_loss = total_loss + loss.item()
    total_acc = total_acc + compute_accuracy(logits,labels)

    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits,axis=1).flatten()
    labels = labels.detach().cpu().numpy().flatten()

    true_labels.extend(labels)
    predictions.extend(logits)

  avg_loss = total_loss/len(test_loader)
  avg_acc = total_acc/len(test_loader)
  macro_f1_score =  classification_report(true_labels, predictions,output_dict=True)['macro avg']['f1-score']

  return avg_loss,avg_acc,macro_f1_score

In [24]:
test_loss,test_acc,f1_score = test_evaluate(model)
print(f'\nTest Accuracy: {test_acc:.3f} | Test Loss: {test_loss:.3f} | F1 Score: {f1_score:.3f} ')


Test Accuracy: 0.789 | Test Loss: 0.451 | F1 Score: 0.787 


#Actual test

In [6]:
df_test_set=pd.read_csv('/content/drive/MyDrive/NLP /Project/H2_Hate_Offensive_Language_Identification_test.csv')

In [7]:
df_test_set['labels']=list(np.ones(df_test_set['text'].values.shape[0]))

In [8]:
X_test_set = list(df_test_set['text'].values)
y_test_set = list(df_test_set['labels'].values)

In [9]:
df_test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844 entries, 0 to 843
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    844 non-null    object 
 1   id      844 non-null    int64  
 2   labels  844 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 19.9+ KB


In [21]:
test_dataset = generate_dataset(X_test_set,y_test_set,tokenizer)
# test_loader = get_data_loader(test_dataset,SequentialSampler)
test_loader=DataLoader(dataset=test_dataset,batch_size=1,shuffle=False)
model = torch.load('/content/drive/MyDrive/NLP /Project/final_english_model_robert_8Sept.pt')
model = model.to(device)

In [27]:
def generate_predictions(model,test_loader):
  model.train()
    
  true_labels = []
  predictions = []
  ids=[]
  for step,batch in enumerate(test_loader):

    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_loader)))

    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
      outputs = model(input_ids,attention_mask = attention_mask,labels = labels.long())
    # import pdb; pdb.set_trace()
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits,axis=1).flatten()
    predictions.extend(logits)
    # ids.extend(input_ids.detach().cpu().numpy())
  df =pd.DataFrame()
  df['label']=predictions
  # df['id']=ids
  return df

In [28]:
predicted_df=generate_predictions(model,test_loader)

  Batch    50  of    844.
  Batch   100  of    844.
  Batch   150  of    844.
  Batch   200  of    844.
  Batch   250  of    844.
  Batch   300  of    844.
  Batch   350  of    844.
  Batch   400  of    844.
  Batch   450  of    844.
  Batch   500  of    844.
  Batch   550  of    844.
  Batch   600  of    844.
  Batch   650  of    844.
  Batch   700  of    844.
  Batch   750  of    844.
  Batch   800  of    844.


In [30]:
predicted_df['id']=list(df_test_set['id'].values)

In [32]:
predicted_df.to_csv('/content/drive/MyDrive/NLP /Project/H2_Hate_Offensive_Language_Identification_test_predictions.csv')